In [6]:
import os
import glob

import pandas as pd
import numpy as np

import cv2

import matplotlib.pyplot as plt

## Load Txt Log

In [105]:
def read_log(test_file):
    columns_labels = ['DateTime','SearchMap','Tag','FileName', 'MsgType', 'BufferLength', 'Focus', 'MapId', 'Pose', 'LocalizationFlag']
    df = pd.DataFrame([],index=[],
               columns=columns_labels)
    idx = 0
    with open(test_file, 'r') as file_to_read:
        while True:
            lines = file_to_read.readline()
            if not lines:
                break
            time = lines[0:19]
            flag = lines[22:34]
            
            # received a new localization requirement
            if(flag == "receive data"):
                idx += 1
                Message = [None for i in range(len(columns_labels))]
                id_tag = lines.find("_tag")
                id_msgType = lines.find("_msgType")
                id_imgsize = lines.find("_imgSize")
                id_param = lines.find("param")
                tag = lines[id_tag+6: id_msgType-1]
                msgType = lines[id_msgType+9: id_imgsize-1]
                imagesize = lines[id_imgsize+9: id_param-2]
                #print(flag, tag, msgType, imagesize)
                
                id_focus = lines.find("focus=")
                focus = lines[id_focus+6: id_focus+8+9]
                
                id_mapid = lines.find("mapId=")
                #mapid_flag = lines[id_mapid+6: id_mapid+7]
                
                Message[0] = time
                Message[2] = tag
                
                if(tag[0:3] == "ios"):
                    image_file = tag + ".jpg"
                else:
                    image_file = tag + ".YUV"
                    
                Message[3] = image_file
                Message[4] = msgType
                Message[5] = int(imagesize)
                Message[6] = float(focus)   
                
                if(lines[id_mapid+6: id_mapid+7] == "," or lines[id_mapid+6: id_mapid+10] == "null"):
                    # ask for the map search
                    lines_s = file_to_read.readline() # should be start search map
                    lines_s = file_to_read.readline() # should be end search map
                    if not lines_s:
                        break
                    id_mapsearchid = lines_s.find("_mapList:")
                    map_search_result = lines_s[id_mapsearchid+10: id_mapsearchid+12]
                    #print(map_search_result)
                    if(map_search_result == "[]"):
                        # search map failed -> move on to next line
                        Message[1] = "Fail"
                    else:
                        Message[1] = "Success"
                else:
                    Message[1] = "FromInput"
                    
                if(Message[1] == "Fail"):
                    df.loc[idx]= Message
                    Message[9] = 'Fail'
                    continue;
                    
                lines_mapid = file_to_read.readline() # should be start location map
                id_mapsearchid = lines_mapid.find("_mapId :")
                Message[7] = lines_mapid[id_mapsearchid+9: -1] 
                
                lines_pose = file_to_read.readline() # should be end location map
                id_pose = lines_pose.find("position=")
                id_mapid = lines_pose.find(", mapId=")
                #print(lines_pose)
                #print(Message[1], lines_pose[id_pose+10:id_mapid-1])
                pose = (lines_pose[id_pose+10:id_mapid-1]).split(',')
                pose = np.asarray(pose, np.float)
                Message[8] = pose
                if(np.sum(pose) == 0):
                    Message[9] = 'Fail'
                else:
                    Message[9] = 'Success'
                
                
                df.loc[idx]= Message
    return df

In [106]:
test_file = "ios-2317e031-13-53/location_log.txt"
df = read_log(test_file)
df.head()

,DateTime,SearchMap,Tag,FileName,MsgType,BufferLength,Focus,MapId,Pose,LocalizationFlag
1,2020-08-12 13:53:11,Fail,ios_1,ios_1.jpg,301,4250,522.217468,None,None,None
2,2020-08-12 13:53:19,Success,ios_2,ios_2.jpg,301,22007,523.052368,e69a4123-d89f-11ea-9cb1-da42307ea98d,"[0.496057, 0.019977, 0.86806, 0.0, -0.004627, ...",Success
3,2020-08-12 13:53:27,FromInput,ios_3,ios_3.jpg,301,19285,515.538086,e69a4123-d89f-11ea-9cb1-da42307ea98d,"[0.486308, 0.006269, 0.873765, 0.0, 0.001378, ...",Success
4,2020-08-12 13:53:35,FromInput,ios_4,ios_4.jpg,301,18177,525.139648,e69a4123-d89f-11ea-9cb1-da42307ea98d,"[0.524659, -0.117385, 0.843181, 0.0, -0.045336...",Success
5,2020-08-12 13:53:48,FromInput,ios_5,ios_5.jpg,301,27908,523.469849,e69a4123-d89f-11ea-9cb1-da42307ea98d,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Fail


In [49]:
test_file = "android-90066be6-14-42/location_log.txt"
df = read_log(test_file)
df.head()

,DateTime,SearchMap,Tag,FileName,MsgType,BufferLength,Focus,MapId,Pose,LocalizationFlag
1,2020-08-12 14:42:58,Success,android_1,android_1.YUV,301,307200,498.02417,e69a4123-d89f-11ea-9cb1-da42307ea98d,"[0.438228, 0.046195, 0.897676, 0.0, -0.002198,...",Success
2,2020-08-12 14:43:13,FromInput,android_2,android_2.YUV,301,307200,498.02417,e69a4123-d89f-11ea-9cb1-da42307ea98d,"[-0.092777, -0.137731, 0.986115, 0.0, -0.01022...",Success
3,2020-08-12 14:43:32,FromInput,android_3,android_3.YUV,301,307200,498.02417,e69a4123-d89f-11ea-9cb1-da42307ea98d,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Fail
4,2020-08-12 14:43:57,FromInput,android_4,android_4.YUV,301,307200,498.02417,e69a4123-d89f-11ea-9cb1-da42307ea98d,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Fail
5,2020-08-12 14:44:13,FromInput,android_5,android_5.YUV,301,307200,498.02417,e69a4123-d89f-11ea-9cb1-da42307ea98d,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Fail


## Analysis Android results

In [47]:
def load_image_scan(file):
    #print(file,file[-3:] )
    if file[-3:] == "YUV":
        scan = np.fromfile(file, dtype=np.uint8).reshape((480,640))
    elif file[-3:] == "jpg":
        scan = cv2.imread(file)
    return scan

In [83]:
def AnalysisLocalizationResult(log_folder):
    test_file = log_folder + "/location_log.txt"
    df = read_log(test_file)
    print("------ Read", df.shape[0], "instance(s), from folder",log_folder,"------")
    
    n_search = df.shape[0] - np.sum(df['SearchMap'] == 'FromInput')
    print("Map Search : Process", n_search, "time(s).")
    
    n_fail = np.sum(df['LocalizationFlag'] == 'Fail')
    n_succ = np.sum(df['LocalizationFlag'] == 'Success')
    print("Localization : Total Success Rate is", n_succ/df.shape[0], " [",n_succ,"/",df.shape[0],"]")

    failed_images = df['FileName'][df['LocalizationFlag'] == 'Fail']
    i = 0
    
    while(i < len(failed_images.values)):
        image_path = failed_images.values[i]
        focus = df['Focus'][failed_images.index[i]]
        image_path_full = log_folder + "/data/" + image_path
        image = load_image_scan(image_path_full)
        image = cv2.putText(image, str(focus)[0:5], (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2)
        cv2.imshow("Failed Image Show", image)
        key = cv2.waitKey(0)
        if key == ord('d'): 
            i += 1
        elif key == ord('a'): 
            i -= 1
            i = max(i,0)
        elif key == ord('q'):
            break
    cv2.destroyAllWindows()

In [89]:
android_log_folders = glob.glob("android*")
for i in range(len(android_log_folders)):
    AnalysisLocalizationResult(android_log_folders[i])

------ Read 94 instance(s), from folder android-8183a6f1-14-18 ------
Map Search : Process 1 time(s).
Localization : Total Success Rate is 0.48936170212765956  [ 46 / 94 ]
------ Read 73 instance(s), from folder android-90066be6-14-42 ------
Map Search : Process 1 time(s).
Localization : Total Success Rate is 0.4246575342465753  [ 31 / 73 ]
------ Read 84 instance(s), from folder android-ec3161d8-15-46 ------
Map Search : Process 1 time(s).
Localization : Total Success Rate is 0.4642857142857143  [ 39 / 84 ]


In [107]:
ios_log_folders = glob.glob("ios*")
for i in range(len(ios_log_folders)):
    AnalysisLocalizationResult(ios_log_folders[i])

------ Read 99 instance(s), from folder ios-2317e031-13-53 ------
Map Search : Process 6 time(s).
Localization : Total Success Rate is 0.5757575757575758  [ 57 / 99 ]
------ Read 65 instance(s), from folder ios-962bdb1b-15-33 ------
Map Search : Process 2 time(s).
Localization : Total Success Rate is 0.5384615384615384  [ 35 / 65 ]
------ Read 70 instance(s), from folder ios-e36d3ae6-15-20 ------
Map Search : Process 2 time(s).
Localization : Total Success Rate is 0.4714285714285714  [ 33 / 70 ]
